## XGBoost

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

https://github.com/dmlc/xgboost/blob/master/python-package/xgboost/sklearn.py

https://github.com/tqchen/xgboost/blob/master/demo/guide-python/README.md

In [27]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [29]:
xgb.XGBClassifier?

xgb.XGBClassifier(self, max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, **kwargs)

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [9]:
X_train = train.drop(['Id','Cover_Type'],1)
Y_train = train['Cover_Type']
X_test = test.drop(['Id'],1)

In [30]:
scaler = StandardScaler()
X_train_tf = pd.DataFrame(scaler.fit_transform(X_train),index=X_train.index, columns=X_train.columns)
X_test_tf = pd.DataFrame(scaler.transform(X_test),index=X_test.index, columns=X_test.columns)

In [123]:
## Led to overfitting
#xgbc=xgb.XGBClassifier(objective='multi:softprob',n_jobs=4,learning_rate=1.2,num_class=7,eval_metric="mlogloss")
#0.97169312169312172
#[ 0.74603175  0.71990741  0.73941799  0.75198413  0.82010582]

#xgbc=xgb.XGBClassifier(objective='multi:softprob',n_jobs=4,learning_rate=1.2,num_class=7,eval_metric="mlogloss",reg_lambda=5)
#0.95932539682539686
#[ 0.74272487  0.72354497  0.73842593  0.7582672   0.81316138]

#xgbc=xgb.XGBClassifier(objective='multi:softmax',n_jobs=4,learning_rate=1.2,num_class=7,eval_metric="mlogloss",reg_lambda=5)
#0.95932539682539686
#[ 0.74272487  0.72354497  0.73842593  0.7582672   0.81316138]

#xgbc=xgb.XGBClassifier(objective='multi:softprob',n_jobs=4,learning_rate=1.2,num_class=7,eval_metric="mlogloss",reg_lambda=10, reg_alpha=0.01)

xgbc=xgb.XGBClassifier(objective='multi:softprob'
                       ,n_jobs=4
                       ,learning_rate=1.2
                       ,num_class=7
                       ,eval_metric="mlogloss"
                       ,reg_lambda=10
                       ,reg_alpha=0.01
                       ,colsample_bytree=0.5
                       ,subsample=0.5
                       ,gamma=0.5
                       ,max_depth=12)

xgbc.fit(X_train_tf,Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, eval_metric='mlogloss', gamma=0.5,
       learning_rate=1.2, max_delta_step=0, max_depth=12,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=4,
       nthread=None, num_class=7, objective='multi:softprob',
       random_state=0, reg_alpha=0.01, reg_lambda=10, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.5)

In [124]:
Y_train_pred = xgbc.predict(X_train_tf)
Y_train_pred

array([5, 5, 2, ..., 3, 3, 3])

In [125]:
skm.accuracy_score(Y_train,Y_train_pred)

0.99484126984126986

In [135]:
# xgbc=xgb.XGBClassifier(objective='multi:softprob'
#                        ,n_jobs=4
#                        ,eta=0.2
#                        ,num_class=7
#                        ,eval_metric="mlogloss"
#                        ,reg_lambda=10
#                        ,reg_alpha=0.01
#                        ,colsample_bytree=0.5
#                        ,subsample=0.5
#                        ,gamma=1.5
#                        ,max_depth=12)

xgbc=xgb.XGBClassifier(objective = "multi:softmax",
                       eval_metric = "merror",
                       max_depth = 12,
                       eta = 0.0399,
                       gamma = 1.2393,
                       subsample = 0.7052,
                       colsample_bytree = 0.6296,
                       min_child_weight = 7,
                       colsample_bylevel = 1,
                       reg_lambda = 1, 
                       reg_alpha = 0, 
                       num_class = 7,
                       booster = "gbtree",
                       silent = 0)
results=cross_val_score(xgbc,X_train,Y_train,cv=5,n_jobs=8)
print(results)
results.mean()

[ 0.7364418   0.72850529  0.76289683  0.77083333  0.83465608]


0.76666666666666672

In [ ]:
N_estimators = [20,40,60,80]
Criterion = ['gini','entropy']
Max_depth = [20,30,40,50,60,70]
i=0
for n_estimator in N_estimators:
    for criterion in Criterion:
        for max_depth in Max_depth:
            results_rf.loc[i,'n_estimator'] = n_estimator
            results_rf.loc[i,'criterion'] = criterion
            results_rf.loc[i,'max_depth'] = max_depth
            r = cross_val_score(RandomForestClassifier(n_estimators=n_estimator,
                                                       criterion=criterion,
                                                      max_depth=max_depth,
                                                      random_state=100),
                               X=X_train_tf,y=Y_train,scoring='accuracy',cv=5,n_jobs=4)
            results_rf.loc[i,'accuracy_min'] = r.min()
            results_rf.loc[i,'accuracy_max'] = r.max()
            results_rf.loc[i,'accuracy_mean'] = r.mean()
            i = i+1